# Using Pandas



In this notebook, we will be looking at pandas, a Python library that provides many useful tools for loading, displaying, and cleaning data. Please have a look at the [official Pandas documentation](https://pandas.pydata.org/docs/reference/index.html) to learn more about any of the functions you encounter in this notebook.

To aid us in showing off the functionality of this library, we will be looking at the MetObjects dataset, which comes courtesy of the [Metropolitan Museum of Art in New York](https://www.metmuseum.org/). This data can be found on GitHub [here](https://github.com/metmuseum/openaccess/tree/master) as well as on [Kaggle](https://www.kaggle.com/metmuseum/the-metropolitan-museum-of-art-open-access).

For this lecture we will download the dataset from my website as a zip file then extract that into a new folder in this current directory.

## Downloading the dataset
You can use a leading `!` in a line of Jupyter notebook code to specify that the rest of the line should be interpreted as a shell command. This is convenient for modifying files or running scripts that live on your filesystem without having to switch between the browser and terminal. Let's use this syntax to create a directory for the Met Museum dataset

In [112]:
data_dir = "./data/met_objects"   # Normal python code


# Jupyter notebook "magic" lines prepended with a ! character
!mkdir -p $data_dir
# note the -p flag in mkdir, which allows us to create a new directory without throwing an error if it already exists
# it will also create any intermediate directories that don't exist

Now we are going to download the dataset from my website and extract it into the data_dir folder. There are many different methods to do this, in this case we are going to use the ```requests``` library to download the file and ```zipfile``` to extract it.

In [113]:
import requests
from tqdm import tqdm
from pathlib import Path

def download(url: str, fname: str):
    resp = requests.get(url, stream=True,verify=False)
    total = int(resp.headers.get('content-length', 0))
    # Can also replace 'file' with a io.BytesIO object
    with open(fname, 'wb') as file, tqdm(
        desc=fname,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as progress_bar:
        for data in resp.iter_content(chunk_size=1024):
            size = file.write(data)
            progress_bar.update(size)


The above function takes two parameters, one is the url of the file to download and the other is the path to save the file. We can use it as follows to download the zip file. 

In [114]:
url="https://nccastaff.bournemouth.ac.uk/jmacey/SEForMedia/DataSets/MetObjects.zip"
zip_file=f"{data_dir}/MetObjects.zip"
csv_file=f"{data_dir}/MetObjects.csv"
# check to see if we have downloaded the data
if not Path(zip_file).exists() or not Path(csv_file).exists():
    download(url, f"{data_dir}/MetObjects.zip")
else:
    print(f"{zip_file} already downloaded")


./data/met_objects/MetObjects.zip already downloaded


Now we will unzip the dowlnoaded file into the data_dir folder. We can use the ```zipfile``` library to do this.

In [115]:
import zipfile
with zipfile.ZipFile(zip_file, "r") as zip_ref:
            print(f"[INFO] Unzipping {zip_file} data...") 
            zip_ref.extractall(data_dir)

[INFO] Unzipping ./data/met_objects/MetObjects.zip data...


We should now be able to see the file MetObject.csv in the data_dir folder.

In [116]:
import os
os.listdir(data_dir)

['MetObjects.zip', 'MetObjects.csv']

## Setting up the environment
Now that we have the data, we can start by importing the necessary libraries and loading the data into a pandas DataFrame. To make displaying the data easier, we can set some options for pandas to display more rows and columns before we create our first data frame.

In [117]:
# Module imports and plot settings
import pandas as pd
import matplotlib.pyplot as plt


pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None 
plt.rcParams["figure.figsize"] = [8, 7]
plt.rcParams["figure.autolayout"] = True

## The DataFrame

The DataFrame is the central data structure provided by Pandas, and it is this structure that we need to interrogate when we want to ask questions about our data. You can think of a DataFrame as a table with rows of records and columns that describe the fields of those records. Pandas provides built in functions for loading text files and automatically puts their contents into a DataFrame. The dataset we just downloaded (`MetObjects.csv`) is a CSV (comma separated value) file, so we need to use the `load_csv` function provided by Pandas.

In [118]:
# Loading the dataset into a DataFrame
# The `sep` argument is the delimiter character, which 
# tells pandas how to separate columns. Be careful: always inspect your CSV 
# files to check what the delimiter character is: sometimes people use
# tabs (\t). Make a note of what the delimiter is and pass it into 
# read_csv. In this case, the separator is a comma (,) so that's what 
# we'll use.
dataset = pd.read_csv(csv_file, sep=',') 
dataset  # <- this is a Pandas DataFrame

/var/folders/gn/c4kfmwrs2ml20f60x0lhnmjw0000gn/T/ipykernel_21899/4259340645.py:8: DtypeWarning: Columns (5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(csv_file, sep=',')


,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1979.486.1,False,False,False,1,NaN,The American Wing,1979.0,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,16429,Maker,,James Barton Longacre,"American, Delaware County, Pennsylvania 1794–1...",,"Longacre, James Barton",American,1794,1869,NaN,http://vocab.getty.edu/page/ulan/500011409,https://www.wikidata.org/wiki/Q3806459,1853,1853,1853,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1979",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/1,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
1,1980.264.5,False,False,False,2,NaN,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,107,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",American,1785,1844,NaN,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648,1901,1901,1901,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
2,67.265.9,False,False,False,3,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
3,67.265.10,False,False,False,4,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4,67.265.11,False,False,False,5,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484951,55.621.134,False,False,False,900605,NaN,Drawings and Prints,1955,Print,Holiday Card,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",American,1913,1985,NaN,NaN,NaN,1944,1939,1949,Engraving,Block: 3 15/16 × 2 1/2 in. (10 × 6.4 cm)\r\nSh...,"Gift of the Estate of John Taylor Arms, 1955",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
484952,1977.646,False,False,False,900606,NaN,Drawings and Prints,1977,Print,Brooklyn Local,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",Americ

## Accessing and displaying data

### Integer indexing
Similar to Python list slices, uses 0-indexed start and end positions to return a subset of the dataframe. With a Padas dataframe, this is done via the `iloc` indexer.

In [119]:
# Get rows number 29 to 35
int_indexing = dataset.iloc[29 : 35]
int_indexing

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
29,16.74.37,False,False,False,30,NaN,The American Wing,1916.0,Centavos,"Coin, 10 Centavos",Mexican,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1885,1885,1885,Nickel,Diam. 3/4 in. (1.9 cm),"Gift of Mrs. Russell Sage, 1916",Made in,NaN,NaN,NaN,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/30,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
30,16.74.40,False,False,False,31,NaN,The American Wing,1916.0,Centavos,"Coin, 10 Centavos",Mexican,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1885,1885,1885,Nickel,Diam. 3/4 in. (1.9 cm),"Gift of Mrs. Russell Sage, 1916",Made in,NaN,NaN,NaN,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/31,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
31,09.9.15,False,False,False,32,NaN,The American Wing,1909.0,Pesos,"Coin, 20 Pesos",Mexican,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1866,1866,1866,Gold,Diam. 2 1/8 in. (5.4 cm),"Gift of H. R. Ickelheimer, 1909",Made in,NaN,NaN,NaN,Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/32,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
32,64.62,False,False,False,33,774,The American Wing,1964.0,Bust,Bust of Abraham Lincoln,American,NaN,NaN,NaN,NaN,16459,Maker,,James Gillinder and Sons,"American, 1861–ca. 1930",,"Gillinder and Sons, James",American,1861,1930,NaN,http://vocab.getty.edu/page/ulan/500524407,https://www.wikidata.org/wiki/Q99264077,1876,1876,1876,Pressed glass,2 3/4 x 3 1/2 x 2 3/4 in. (7 x 8.9 x 7 cm),"Gift of Teunis G. B. Cortelyou, 1964",Made in,Philadelphia,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/33,https://www.wikidata.org/wiki/Q116250677,NaN,"Metropolitan Museum of Art, New York, NY",Men|Abraham Lincoln|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
33,1970.289.6,False,False,True,34,774,The American Wing,1970.0,Clock,Acorn Clock,American,NaN,NaN,NaN,NaN,108,Maker,,Forestville Manufacturing Company,1835–1853,,Forestville Manufacturing Company,American,1835,1853,NaN,NaN,NaN,1847–50,1847,1850,"Mahogany, laminated",24 3/8 x 14 5/8 x 5 1/8 in. (61.9 x 37.1 x 13 cm),"Gift of Mrs. Paul Moore, 1970",Made in,Bristol,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/34,https://www.wikidata.org/wiki/Q116373732,NaN,"Metropolitan Museum of Art, New York, NY",Landscapes|Boats,http://vocab.getty.edu/page/aat/300132294|http...,https://www.wikidata.org/wiki/Q191163|https://...
34,04.1a–c,True,True,False,35,706,The American Wing,1904.0,Vase,The Adams Vase,American,NaN,NaN,NaN,NaN,108316253,Designer|Manufacturer,Designed by|Manufactured by,Paulding Farnham|Tiffany & Co.,1859–1927|1837–present,|,"Farnham, Paulding|Tiffany & Co.",American|,1859 |1837,1927 |9999,|,http://vocab.getty.edu/page/ulan/500336597|htt...,https://www.wikidata.org/wiki/Q13476260|https:...,1893–95,1893,1895,"Gold, amethysts, spessartites, tourmalines, fr...",Overall: 19 7/16 x 13 x 9 1/4 in. (49.4 x 33 x...,"Gift of Edward D. Adams, 1904",Made in,New York,NaN,NaN,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum

### Boolean Series
A series is a 1-D array - a boolean series is one that is filled with boolean (i.e., `True` or `False`) values. We can pass boolean series into a Dataframe's `loc` indexer to keep only the values that align with `True`. Different boolean series of the same length can be combined using the following logical operators: `&` (and), `|` (or), `~` (not).

In [120]:
# getting all rows from the 'Medieval Art' department
medieval_art_bool_series = dataset['Department'] == "Medieval Art"
dataset.loc[medieval_art_bool_series]

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
21606,28.99.60,False,False,True,26491,NaN,Medieval Art,1928.0,Fragment,Fragment,Syrian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13th century,1201,1300,Glass,L. 1 1/2 in. (3.8 cm),"Gift of Clarence H. Mackay, Archer M. Huntingt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glass-Vessels,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
21607,28.99.61a,False,False,True,26492,NaN,Medieval Art,1928.0,Lamp fragment,Lamp Fragment,Syrian or Crusader,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13th century,1201,1300,Glass,(a) L. 2 in. (5.1 cm),"Gift of Clarence H. Mackay, Archer M. Huntingt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glass-Vessels,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
21608,28.99.62,False,False,True,26493,NaN,Medieval Art,1928.0,Neck of flask,Neck of Flask,Syrian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13th century,1201,1300,Glass,L. 2 1/4 in. (5.7 cm),"Gift of Clarence H. Mackay, Archer M. Huntingt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glass-Vessels,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
21609,28.99.63,False,False,True,26494,NaN,Medieval Art,1928.0,Fragment of a bowl,Fragment of a Bowl,French or German,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13th century,1201,1300,Glass,L. 1 3/4 in. (4.5 cm),"Gift of Clarence H. Mackay, Archer M. Huntingt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glass-Vessels,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
21610,28.99.64,False,False,True,26495,NaN,Medieval Art,1928.0,Foot of a vessel,Foot of a Vessel,Syrian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13th century,1201,1300,Glass,L. 1 1/2 in. (3.8 cm),"Gift of Clarence H. Mackay, Archer M. Huntingt...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glass-Vessels,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474485,2020.255,False,False,True,841757,307.0,Medieval Art,2020.0,Pedestal,Pedestal,North or Central European,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15th century,1400,1500,Wrought Iron,"41 3/4 × 22 × 22 in., 196.2 lb. (106 × 55.9 × ...","Purchase, Michael and Patricia O'Neill Gift, 2020",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Metalwork-Iron,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q116445503,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
475103,2021.418,False,False,True,844002,304.0,Medieval Art,2021.0,Game Piece,Game Piece with Stork and Fox,North French or German,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12th century,1100,1200,Walrus ivory,Diameter: 2 1/4 in. (5.7 cm); thickness 1.1 cm,"Pfeiffer Fund, 2020",NaN,NaN,NaN,NaN,NaN,

In [121]:
# getting rows from the 'Medieval Art' OR 'European Sculpture and Decorative Arts' departments
esada_bool_series = dataset['Department'] == "European Sculpture and Decorative Arts"
both = esada_bool_series | medieval_art_bool_series
dataset.loc[both]

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
4843,10.125.583o,False,False,False,5209,774,European Sculpture and Decorative Arts,1910.0,Curtain knob,Curtain knob,probably American,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700–1900,1700,1900,Brass,Diameter: 2 1/8 in. (5.4 cm),"Gift of Mrs. Russell Sage, 1909",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Metalwork-Brass,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q116392256,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4844,10.125.583p,False,False,False,5210,NaN,European Sculpture and Decorative Arts,1910.0,Curtain knob,Curtain knob,probably American,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700–1900,1700,1900,Brass,Diameter: 2 1/8 in. (5.4 cm),"Gift of Mrs. Russell Sage, 1909",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Metalwork-Brass,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Eagles,http://vocab.getty.edu/page/aat/300250049,https://www.wikidata.org/wiki/Q2092297
7338,06.719,False,False,True,7829,NaN,European Sculpture and Decorative Arts,1906.0,Spoon,Spoon,Flemish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700–1800,1700,1800,Pewter,No dimensions recorded,"Rogers Fund, 1906",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Metalwork-Pewter,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
12564,1984.331.25,False,False,True,13737,NaN,European Sculpture and Decorative Arts,1984.0,Embroidered picture,Embroidered Picture,British,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ca. 1845,1842,1845,Embroidered wool on linen,22 3/4 x 18 1/2 in. (57.8 x 47 cm),"Gift of Barbara Schiff Sinauer, 1984",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Textiles,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Flowers,http://vocab.getty.edu/page/aat/300132399,https://www.wikidata.org/wiki/Q506
12567,38.95,False,False,True,13740,NaN,European Sculpture and Decorative Arts,1938.0,Embroidered picture,Embroidered picture of Charles I,British,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1650–75,1650,1675,Canvas embroidered with wool and silk thread,14 1/2 x 10 in. (36.8 x 25.4 cm),"Rogers Fund, 1938",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Textiles,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483238,2022.462.1–.6,False,False,False,894329,NaN,European Sculpture and Decorative Arts,2022,NaN,Group of six examples of European quilt-work o...,"English, French",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,various dates in 18th century,1700,1875,"Cotton, linen",see attached list,"Gift of Karen B. Cohen, in honor of Rochelle C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Textiles-Embroidered,NaN,http://www.metmuseum.org/art/

### Grouping by column name(s)
We can also group the data by a list of columns. This returns a Pandas GroupBy object, which contains a dictionary of mappings from each group name to a Series of its elements

In [122]:
g = dataset.groupby(['Object Name', 'Culture'])
g = g.size().reset_index(name='Counts')
g

,Object Name,Culture,Counts
0,"""Autophone"" Organette",American,1
1,"""Basso""",Italian,1
2,"""Chanot Model"" Violin",possibly French,1
3,"""Humantone"" Nose Flute",American,1
4,"""Japanese Fiddle""",American,1
...,...,...,...
42911,印籠刻昆虫図螺鈿据文象嵌鞘打刀拵 Blade and mounting for a swor...,Japanese,1
42912,富嶽雲烟図小柄 Knife handle (<i>Kozuka</i>) with blade,Japanese,1
42913,秋草に鹿図鐔 Sword guard (<i>Tsuba</i>),Japanese,1
42914,藻に白魚図小柄 Knife handle (<i>Kozuka</i>),Japanese,1


## Using ```where``` to filter data

Similar to Numpy arrays, Pandas dataframes also make use of the ```where``` function to conditionally modify its elements based on some criteria. ```where``` takes a dataframe condition as an argument and returns the modified dataframe - if the condition is fulfilled, it keeps the value of the field, if not, it replaces it with `NaN`. We can use this function to remove objects that do not 

In [123]:
g.where(g['Counts']>1)

,Object Name,Culture,Counts
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
42911,NaN,NaN,NaN
42912,NaN,NaN,NaN
42913,NaN,NaN,NaN
42914,NaN,NaN,NaN


As you can see, most of the records are now NaN. We will look at how these types of situations can be fixed in the upcoming cells. Let's  print out the original dataset again so we can use it as a visual reference for the future:

In [124]:
dataset

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1979.486.1,False,False,False,1,NaN,The American Wing,1979.0,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,16429,Maker,,James Barton Longacre,"American, Delaware County, Pennsylvania 1794–1...",,"Longacre, James Barton",American,1794,1869,NaN,http://vocab.getty.edu/page/ulan/500011409,https://www.wikidata.org/wiki/Q3806459,1853,1853,1853,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1979",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/1,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
1,1980.264.5,False,False,False,2,NaN,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,107,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",American,1785,1844,NaN,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648,1901,1901,1901,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
2,67.265.9,False,False,False,3,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
3,67.265.10,False,False,False,4,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4,67.265.11,False,False,False,5,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484951,55.621.134,False,False,False,900605,NaN,Drawings and Prints,1955,Print,Holiday Card,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",American,1913,1985,NaN,NaN,NaN,1944,1939,1949,Engraving,Block: 3 15/16 × 2 1/2 in. (10 × 6.4 cm)\r\nSh...,"Gift of the Estate of John Taylor Arms, 1955",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
484952,1977.646,False,False,False,900606,NaN,Drawings and Prints,1977,Print,Brooklyn Local,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",Americ

## Data cleaning
Before data can be fed into your application, it needs to be verified and checked for consistency. We can see that there are several problems with the dataset right off the bat:
1. First row seems to contain garbage: none of the column names match up with the data types, and many are NaN
2. It looks like many of the columns are completely empty - they add nothing to the dataset but clutter it
3. Too many columns! This depends on what your needs are, but we don't need all of them for this exercise
4. Inconsistent formatting in the Dimensions column - makes it difficult to use them downstream
5. Mixed datatypes in Year fields

Let's address all of these issues one by one

### Deleting rows by index
We can get rid of the first row (index 0) by taking a slice of the dataframe beginning at index 1 and going all the way to the end.

In [125]:
# Get rid of the first row, we can use dataframe slicing to accomplish this:
dataset = dataset.iloc[1:]
dataset


,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
1,1980.264.5,False,False,False,2,NaN,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,107,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",American,1785,1844,NaN,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648,1901,1901,1901,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
2,67.265.9,False,False,False,3,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
3,67.265.10,False,False,False,4,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4,67.265.11,False,False,False,5,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
5,67.265.12,False,False,False,6,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/6,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484951,55.621.134,False,False,False,900605,NaN,Drawings and Prints,1955,Print,Holiday Card,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",American,1913,1985,NaN,NaN,NaN,1944,1939,1949,Engraving,Block: 3 15/16 × 2 1/2 in. (10 × 6.4 cm)\r\nSh...,"Gift of the Estate of John Taylor Arms, 1955",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
484952,1977.646,False,False,False,900606,NaN,Drawings and Prints,1977,Print,Brooklyn Local,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",American,1913,1985,NaN,NaN,NaN,1944,1939,1949,Etching,Plate: 8 1/4 × 6 7/8 in. (20.9 × 17.5 cm)\r\nS...,"John B. Turner Fund, 1977",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

### Removing columns

Columns can be removed conditionally by checking their contents to see if they meet a certain criteria, or simply by name

In [126]:
# Get rid of all columns that are completely empty
dataset = dataset.dropna(how='all', axis=1) # how=all means drop this key if all items are NaN. Axis=1 means work on columns
dataset


,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Repository,Tags,Tags AAT URL,Tags Wikidata URL
1,1980.264.5,False,False,False,2,NaN,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,107,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",American,1785,1844,NaN,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648,1901,1901,1901,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
2,67.265.9,False,False,False,3,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
3,67.265.10,False,False,False,4,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4,67.265.11,False,False,False,5,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
5,67.265.12,False,False,False,6,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/6,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484951,55.621.134,False,False,False,900605,NaN,Drawings and Prints,1955,Print,Holiday Card,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",American,1913,1985,NaN,NaN,NaN,1944,1939,1949,Engraving,Block: 3 15/16 × 2 1/2 in. (10 × 6.4 cm)\r\nSh...,"Gift of the Estate of John Taylor Arms, 1955",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
484952,1977.646,False,False,False,900606,NaN,Drawings and Prints,1977,Print,Brooklyn Local,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",American,1913,1985,NaN,NaN,NaN,1944,1939,1949,Etching,Plate: 8 1/4 × 6 7/8 in. (20.9 × 17.5 cm)\r\nS...,"John B. Turner Fund, 1977",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/ar

We can also remove columns by name. This is done by passing a list of column names to the `drop` function, along with the `axis=1` argument to specify that we are dropping columns, not rows. We can see all of the keys in the dataframe by calling the `keys` function on it.

In [127]:
dataset.keys()

Index(['Object Number', 'Is Highlight', 'Is Timeline Work', 'Is Public Domain', 'Object ID', 'Gallery Number', 'Department', 'AccessionYear', 'Object Name', 'Title', 'Culture', 'Period', 'Dynasty', 'Reign', 'Portfolio', 'Constituent ID', 'Artist Role', 'Artist Prefix', 'Artist Display Name', 'Artist Display Bio', 'Artist Suffix', 'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date', 'Artist End Date', 'Artist Gender', 'Artist ULAN URL', 'Artist Wikidata URL', 'Object Date', 'Object Begin Date', 'Object End Date', 'Medium', 'Dimensions', 'Credit Line', 'Geography Type', 'City', 'State', 'County', 'Country', 'Region', 'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification', 'Rights and Reproduction', 'Link Resource', 'Object Wikidata URL', 'Repository', 'Tags', 'Tags AAT URL', 'Tags Wikidata URL'], dtype='object')

In [128]:
# At this point, we can also drop the columns that are irrelevant to our needs
# I want these ones:
keep=["Object Number",	
      "Is Public Domain",
      "Department",
      "AccessionYear",	
      "Object Name",	
      "Title",
      "Object Begin Date",	
      "Medium",	
      "Dimensions",	
      "Tags"]

# I can then get the rest of the columns by excluding the ones I want to keep
exclude_cols=[col for col in dataset.keys() if col not in keep]
print(exclude_cols)
# then filter out our dataset
dataset = dataset.drop(exclude_cols, axis=1) # again axis=1 means we work on the columns of this dataframe
dataset

['Is Highlight', 'Is Timeline Work', 'Object ID', 'Gallery Number', 'Culture', 'Period', 'Dynasty', 'Reign', 'Portfolio', 'Constituent ID', 'Artist Role', 'Artist Prefix', 'Artist Display Name', 'Artist Display Bio', 'Artist Suffix', 'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date', 'Artist End Date', 'Artist Gender', 'Artist ULAN URL', 'Artist Wikidata URL', 'Object Date', 'Object End Date', 'Credit Line', 'Geography Type', 'City', 'State', 'County', 'Country', 'Region', 'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification', 'Rights and Reproduction', 'Link Resource', 'Object Wikidata URL', 'Repository', 'Tags AAT URL', 'Tags Wikidata URL']


,Object Number,Is Public Domain,Department,AccessionYear,Object Name,Title,Object Begin Date,Medium,Dimensions,Tags
1,1980.264.5,False,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,1901,Gold,Dimensions unavailable,NaN
2,67.265.9,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),NaN
3,67.265.10,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),NaN
4,67.265.11,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),NaN
5,67.265.12,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),NaN
...,...,...,...,...,...,...,...,...,...,...
484951,55.621.134,False,Drawings and Prints,1955,Print,Holiday Card,1939,Engraving,Block: 3 15/16 × 2 1/2 in. (10 × 6.4 cm)\r\nSh...,NaN
484952,1977.646,False,Drawings and Prints,1977,Print,Brooklyn Local,1939,Etching,Plate: 8 1/4 × 6 7/8 in. (20.9 × 17.5 cm)\r\nS...,NaN
484953,33.40.1,False,Drawings and Prints,1933,Print,Yesterday and Today,1920,Etching,Plate: 8 3/4 × 5 7/8 in. (22.2 × 15 cm)\r\nShe...,NaN
484954,170.1 C42,False,The Libraries,NaN,NaN,"De la loi du contraste simultané des couleurs,...",1839,NaN,"2 pages, 30 unnumbered leaves of plates (some ...",NaN


### Default values
Artists are known to often leave their work untitled. In our dataset, this is not handled very gracefully - the titles of such artworks are simple NaN. Fortunately, we have another way of dealing with missing data: assigning a default value. We can replace any instance of a NaN title with the string "Untitled"

In [129]:
# Setting default values. 

dataset['Title'].fillna('Untitled', inplace=True) # This replaces any instance of NaN in the Title column with Untitled
dataset

,Object Number,Is Public Domain,Department,AccessionYear,Object Name,Title,Object Begin Date,Medium,Dimensions,Tags
1,1980.264.5,False,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,1901,Gold,Dimensions unavailable,NaN
2,67.265.9,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),NaN
3,67.265.10,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),NaN
4,67.265.11,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),NaN
5,67.265.12,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),NaN
...,...,...,...,...,...,...,...,...,...,...
484951,55.621.134,False,Drawings and Prints,1955,Print,Holiday Card,1939,Engraving,Block: 3 15/16 × 2 1/2 in. (10 × 6.4 cm)\r\nSh...,NaN
484952,1977.646,False,Drawings and Prints,1977,Print,Brooklyn Local,1939,Etching,Plate: 8 1/4 × 6 7/8 in. (20.9 × 17.5 cm)\r\nS...,NaN
484953,33.40.1,False,Drawings and Prints,1933,Print,Yesterday and Today,1920,Etching,Plate: 8 3/4 × 5 7/8 in. (22.2 × 15 cm)\r\nShe...,NaN
484954,170.1 C42,False,The Libraries,NaN,NaN,"De la loi du contraste simultané des couleurs,...",1839,NaN,"2 pages, 30 unnumbered leaves of plates (some ...",NaN


In [130]:
# We can show all the rows that have the title 'Untitled' by using boolean indexing
untitled_rows = dataset[dataset['Title'] == 'Untitled']
untitled_rows

,Object Number,Is Public Domain,Department,AccessionYear,Object Name,Title,Object Begin Date,Medium,Dimensions,Tags
30295,96.14.193,True,Asian Art,1896.0,Piece,Untitled,1700,Leather,8 1/10 x 3 5/8 in. (20.6 x 9.2 cm),Animals|Leaves
30296,96.14.1896,True,Asian Art,1896.0,Panel,Untitled,1650,Paint; on leather,9 1/4 x 5 3/8 in. (23.5 x 13.7 cm),Musical Instruments|Men|Elephants|Flowers
30298,12.37.135,False,Asian Art,1912.0,Hanging scroll,Untitled,1732,Hanging scroll; ink and color on silk,67 x 38 in. (170.2 x 96.5 cm),NaN
30335,13.100.112,True,Asian Art,1913.0,Fan mounted as an album leaf,Untitled,1200,Fan mounted as an album leaf; ink and color on...,9 3/8 x 9 3/5 in. (23.8 x 24.4 cm),Narcissus
30338,13.100.124,True,Asian Art,1913.0,Fan mounted as an album leaf,Untitled,1167,Fan mounted as an album leaf; ink on silk,9 3/5 x 9 3/8 in. (24.4 x 23.8 cm),Gibbons
...,...,...,...,...,...,...,...,...,...,...
483684,68.696,False,Drawings and Prints,1968,Print,Untitled,1920,Lithograph,Sheet: 9 7/8 × 12 15/16 in. (25.1 × 32.8 cm),NaN
483687,66.684.1,False,Drawings and Prints,1966,Print,Untitled,1948,Etching,Plate: 9 5/16 × 6 15/16 in. (23.7 × 17.7 cm)\r...,NaN
483688,66.684.2,False,Drawings and Prints,1966,Print,Untitled,1948,Etching,Plate: 9 5/16 × 6 15/16 in. (23.7 × 17.7 cm)\r...,NaN
484251,2023.194,True,Asian Art,2023,Paperweight,Untitled,1200,"Silver, lead",H. 3 1/8 in. (7.9 cm); W. 1 1/4 in. (3.2); L. ...,Horses


### Removing rows
We can also get rid of rows that do not meet certain criteria. For example, given a subset of fields that we deem very important, we can drop all rows are NaN in any of these fields

In [131]:
# Dropping records (rows)
important_cols = ['Tags', 'Dimensions', 'Object Begin Date', 'AccessionYear'] 
dataset = dataset.dropna(subset=important_cols)
dataset

# Note that doing this cut the size of our dataset in half! Compare the number of rows in the previous cell to this one.

,Object Number,Is Public Domain,Department,AccessionYear,Object Name,Title,Object Begin Date,Medium,Dimensions,Tags
6,67.265.13,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),Birds|Coins
7,67.265.14,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,Gold,Diam. 11/16 in. (1.7 cm),Eagles|Men|Profiles
32,64.62,False,The American Wing,1964.0,Bust,Bust of Abraham Lincoln,1876,Pressed glass,2 3/4 x 3 1/2 x 2 3/4 in. (7 x 8.9 x 7 cm),Men|Abraham Lincoln|Portraits
33,1970.289.6,True,The American Wing,1970.0,Clock,Acorn Clock,1847,"Mahogany, laminated",24 3/8 x 14 5/8 x 5 1/8 in. (61.9 x 37.1 x 13 cm),Landscapes|Boats
34,04.1a–c,False,The American Wing,1904.0,Vase,The Adams Vase,1893,"Gold, amethysts, spessartites, tourmalines, fr...",Overall: 19 7/16 x 13 x 9 1/4 in. (49.4 x 33 x...,Animals|Garlands|Birds|Men
...,...,...,...,...,...,...,...,...,...,...
484864,49.90.32,False,Drawings and Prints,1949,Print,Corner Pocket,1926,Etching,Plate: 7 3/4 × 10 1/8 in. (19.7 × 25.7 cm)\r\n...,Billiards
484868,49.90.39,False,Drawings and Prints,1949,Print,Polo,1925,Etching,Plate: 5 15/16 × 6 15/16 in. (15.1 × 17.6 cm)\...,Horses|Sports
484900,49.90.8,False,Drawings and Prints,1949,Print,"The Cat, from ""Night Series""",1925,Lithograph,Plate: 4 1/2 × 5 15/16 in. (11.4 × 15.1 cm)\r\...,Cats
484901,49.90.7,False,Drawings and Prints,1949,Print,"The Cat, from ""Night Series""",1925,Etching,Plate: 4 15/16 × 5 15/16 in. (12.5 × 15.1 cm)\...,Cats


### Data types
We change some data types that don't really make sense: `AccessionYear` and `Object Begin Date` were originally loaded in with mixed datatypes (some are strings, some are numbers), which makes it difficult to sort correctly.

We can convert this data using the pd.to_datetime function, which will convert the data to a datetime object. This will allow us to sort the data correctly. In this case I will convert the `AccessionYear` and `Object Begin Date` columns to datetime objects. I will also ignore any errors that may arise from this conversion and set the `coerce` parameter to True. This will replace any errors with NaT values. I am only intereseed in the year, so I will extract that from the datetime object using the format string '%Y'.

In [132]:

# convert the AccessionYear column to a datetime object
dataset["AccessionYear"] = pd.to_datetime(dataset["AccessionYear"], errors='coerce', format='%Y')
dataset["Object Begin Date"] = pd.to_datetime(dataset["AccessionYear"], errors='coerce', format='%Y')

dataset

,Object Number,Is Public Domain,Department,AccessionYear,Object Name,Title,Object Begin Date,Medium,Dimensions,Tags
6,67.265.13,False,The American Wing,NaT,Coin,Two-and-a-Half Dollar Coin,NaT,Gold,Diam. 11/16 in. (1.7 cm),Birds|Coins
7,67.265.14,False,The American Wing,NaT,Coin,Two-and-a-Half Dollar Coin,NaT,Gold,Diam. 11/16 in. (1.7 cm),Eagles|Men|Profiles
32,64.62,False,The American Wing,NaT,Bust,Bust of Abraham Lincoln,NaT,Pressed glass,2 3/4 x 3 1/2 x 2 3/4 in. (7 x 8.9 x 7 cm),Men|Abraham Lincoln|Portraits
33,1970.289.6,True,The American Wing,NaT,Clock,Acorn Clock,NaT,"Mahogany, laminated",24 3/8 x 14 5/8 x 5 1/8 in. (61.9 x 37.1 x 13 cm),Landscapes|Boats
34,04.1a–c,False,The American Wing,NaT,Vase,The Adams Vase,NaT,"Gold, amethysts, spessartites, tourmalines, fr...",Overall: 19 7/16 x 13 x 9 1/4 in. (49.4 x 33 x...,Animals|Garlands|Birds|Men
...,...,...,...,...,...,...,...,...,...,...
484864,49.90.32,False,Drawings and Prints,1949-01-01,Print,Corner Pocket,1949-01-01,Etching,Plate: 7 3/4 × 10 1/8 in. (19.7 × 25.7 cm)\r\n...,Billiards
484868,49.90.39,False,Drawings and Prints,1949-01-01,Print,Polo,1949-01-01,Etching,Plate: 5 15/16 × 6 15/16 in. (15.1 × 17.6 cm)\...,Horses|Sports
484900,49.90.8,False,Drawings and Prints,1949-01-01,Print,"The Cat, from ""Night Series""",1949-01-01,Lithograph,Plate: 4 1/2 × 5 15/16 in. (11.4 × 15.1 cm)\r\...,Cats
484901,49.90.7,False,Drawings and Prints,1949-01-01,Print,"The Cat, from ""Night Series""",1949-01-01,Etching,Plate: 4 15/16 × 5 15/16 in. (12.5 × 15.1 cm)\...,Cats


You can see some of the data has NaT values. We can drop these for our final dataset as follows


In [133]:
dataset=dataset.dropna(subset=["AccessionYear"])
dataset=dataset.dropna(subset=["Object Begin Date"])
dataset

,Object Number,Is Public Domain,Department,AccessionYear,Object Name,Title,Object Begin Date,Medium,Dimensions,Tags
163844,33.43.27,False,Photographs,1933-01-01,Photograph,"Mary and Her Mother, Long Island",1933-01-01,Direct carbon print,Image: 34.8 x 27.5cm (13 11/16 x 10 13/16in.)\...,Infants|Mothers
163847,33.43.272,False,Photographs,1933-01-01,Photograph,Mein Garten,1933-01-01,Gum bichromate over platinum print,32.7 x 44.7 cm. (12 7/8 x 17 5/8 in.),Gardens|Girls|Women
163848,33.43.273,False,Photographs,1933-01-01,Photograph,Teestilleben,1933-01-01,Gum bichromate print,28.3 x 38.3 cm. (11 1/8 x 15 1/16 in.),Cups|Drinking Glasses|Saucers|Teapots
163852,33.43.278,False,Photographs,1933-01-01,Photograph,Ein Sommertag,1933-01-01,Gum bichromate print,78.7 x 52.6 cm. (31 x 20 11/16 in.),Landscapes|Clouds
163853,33.43.279,False,Photographs,1933-01-01,Photograph,Sirócco,1933-01-01,Gum bichromate print,55.2 x 73.7 cm (21 3/4 x 29 in.),Landscapes|Trees|Clouds
...,...,...,...,...,...,...,...,...,...,...
484864,49.90.32,False,Drawings and Prints,1949-01-01,Print,Corner Pocket,1949-01-01,Etching,Plate: 7 3/4 × 10 1/8 in. (19.7 × 25.7 cm)\r\n...,Billiards
484868,49.90.39,False,Drawings and Prints,1949-01-01,Print,Polo,1949-01-01,Etching,Plate: 5 15/16 × 6 15/16 in. (15.1 × 17.6 cm)\...,Horses|Sports
484900,49.90.8,False,Drawings and Prints,1949-01-01,Print,"The Cat, from ""Night Series""",1949-01-01,Lithograph,Plate: 4 1/2 × 5 15/16 in. (11.4 × 15.1 cm)\r\...,Cats
484901,49.90.7,False,Drawings and Prints,1949-01-01,Print,"The Cat, from ""Night Series""",1949-01-01,Etching,Plate: 4 15/16 × 5 15/16 in. (12.5 × 15.1 cm)\...,Cats


## Dataframe interrogation
We can now begin to ask some interesting questions about this dataset:
1. Which department houses the oldest artwork in the museum? Use `Object Begin Date` for this task.
2. What is the proportion of artworks from each department? Display this graphically using [`pd.DataFrame.plot.pie`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.pie.html)
3. _What is the most common theme across all the paintings? Or, which tag is most common?**\*\***_  

_**\*\* Slightly more difficult, multi-step problem**_

## The oldest artwork

To do this we can sort the data by the "Object Begin Date" column and then take the first row. We can then print out the department that houses this artwork.

In [135]:
# Put code in here


The oldest record in the dataset is from the Greek and Roman Art department


<details>

<summary>Solution</summary>

```python
dataset_sorted = dataset.sort_values('Object Begin Date')
oldest_record = dataset_sorted.iloc[0] # access the oldest record
department_name = oldest_record['Department'] # get the name of the Department from the record
print(f"The oldest record in the dataset is from the {department_name} department")
```

</details>




## Proportion of artworks by department

In this case we can use the groupby function to grab the departments and then count the number of records in each department. We can then use the plot function to display this data as a pie chart.

In [140]:
# put code here

<details>

<summary>Solution</summary>

```python

departments = dataset.groupby(['Department']).size().reset_index(name='Counts')
departments

```

</details>

We can use the build in function plot.pie to display this data as a pie chart

In [ ]:
# put code  here

<details>

<summary>Solution</summary>

```python
# The DataFrame has a built-in plotting function that you can 
# call like so: 
departments.plot.pie(y="Counts", 
                     explode=[0.125 for _ in range(len(departments))], 
                     labels=departments['Department'], 
                     legend=None, ylabel="")
```

</details>





## Most common theme

This is a multi stage process,  first we are going to generate an empty list to store all the tags in.
New we will generate a function to split the tags into individual tags and add them to the list. We can then apply this function to the tags column of the dataset.

Note the tags are separated by a pipe character `|`. We can use the `str.split` function to split the tags into a list of tags. 

Now we can use the power of pandas to generate a new dataframe from the tags list and run some analysis on it.


In [143]:
# write code here

<details>

<summary>Solution</summary>

```python
# declare an empty list to collect all the separated tags in the database
tags_list = []   
# get the Tags series out of the dataset and assign it to a variable
# called tags_series
tags_series = dataset['Tags']


# Write a function that accepts a Tag string and a list, and updates 
# the list with the individual tags found in the string.
def update_list(t, l):
    separated = t.split("|")
    l += separated
    
tags_series.apply(update_list, args=(tags_list,))

# turn the tags list into a dataframe called tags_df
tags_df = pd.DataFrame(tags_list, columns=["Tag"])

# group the dataframe by Tag and count the occurences of each tag
tags_df = tags_df.groupby("Tag").size().reset_index(name='Counts')

# sort the list by count to find the one with the tag with the 
# highest number of occurences
tags_df = tags_df.sort_values('Counts', ascending=False)
tags_df.iloc[0]['Tag']
tags_df
```

</details>

